In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import re
import os
import gc

In [ ]:
from module.clean_text import clean_text
from module.add_to_df import add_to_df
from module.save_to_csv import save_to_csv

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
def extract_links(navLinks):
    
    for navlink in navLinks:
        print(f"Processing navlink: {navlink}")
        driver = None
        try:
            driver = webdriver.Chrome()
            
            try:
                driver.get(navlink)
                driver.implicitly_wait(10)  
            except Exception as e:
                print(f"Error accessing navlink {navlink}: {e}")
                continue
            
            try:
                # news_date = driver.find_element(By.CLASS_NAME, 'ap__headerDate').text
                news_date = news_date.split(',')[0]
                news_date = news_date.split(' ')
                # news_date = f'{news_date[1]+' '+ news_date[0]}'
                # year= news_date[2]
            except Exception as e:
                print(f"Error extracting news date from {navlink}: {e}")
                news_date = "Unknown"
            
            try:
                main_body = driver.find_element(By.TAG_NAME, "main")
                find_link = main_body.find_elements(By.TAG_NAME, "a")
                find_link = list(set(find_link))
                
                links = []
                for link in find_link:
                    try:
                        href = link.get_attribute('href')
                        links.append(href)
                    except Exception as e:
                        print(f"Error getting href attribute: {e}")
                        continue
                
                print(links)
                
                story_link = [link for link in links if link is not None and 'story' in link]
                print(len(story_link))
                
                story_link = list(set(story_link))
                print(len(story_link))
                
            except Exception as e:
                print(f"Error extracting story links from {navlink}: {e}")
                story_link = []

            for link in story_link:
                try:
                    driver.get(link)
                    
                    # Wait for the page to load
                    driver.implicitly_wait(10)  
                    
                    # Check if the news is of today
                    try:
                        fulltime = driver.find_element(By.CLASS_NAME, 'date').text
                        # print(fulltime)
                        
                        
                        time = fulltime.split(',')[0]
                        
                        # print(time)
                        
                        takingyear = fulltime.split(',')[1]
                        # print(f'takingyear {takingyear}')
                        year= takingyear.split(' ')[1]
                        # print(f' year {year}')
                    except Exception as e:
                        print(f"Error extracting time from {link}: {e}")
                        time = "Unknown"

                    try:
                        ap_news_content = driver.find_element(By.CLASS_NAME, 'ap__news-content')
                        
                        try:
                            title = ap_news_content.find_element(By.CLASS_NAME, 'news__title')
                            title = clean_text(title.text)
                            print(title)
                        except Exception as e:
                            print(f"Error extracting title from {link}: {e}")
                            title = "No title found"
                        
                        try:
                            news_details = ap_news_content.find_element(By.CLASS_NAME, 'news__details')
                            description = clean_text(news_details.text)
                            print(description)
                        except Exception as e:
                            print(f"Error extracting description from {link}: {e}")
                            description = "No description found"
                        
                        print('\n\n')
                        
                        try:
                            df = add_to_df(title, description)
                            save_to_csv(df, f'/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/AnnapurnaPost/{year}', f'{time}_annupurna_post')
                        except Exception as e:
                            print(f"Error saving data to CSV for {link}: {e}")
                            
                    except Exception as e:
                        print(f"Error finding news content in {link}: {e}")
                        continue
                        
                except Exception as e:
                    print(f"Error processing story link {link}: {e}")
                    continue

        except Exception as e:
            print(f"Error initializing driver for navlink {navlink}: {e}")
            
        finally:
            # Ensure driver is closed even if errors occur
            if driver:
                try:
                    driver.close()
                except Exception as e:
                    print(f"Error closing driver: {e}")
                try:
                    driver.quit()  # Added quit() for complete cleanup
                except Exception as e:
                    print(f"Error quitting driver: {e}")
            
            try:
                gc.collect()
            except Exception as e:
                print(f"Error during garbage collection: {e}")
                
            print("Scraping completed for this navlink.")

In [ ]:
navLinks=[
]
for i in range(2521,2786):
   navLinks.append(f'https://annapurnapost.com/category/politics/?page={i}')
print(navLinks)
extract_links(navLinks)